In [1]:
import numpy as np
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.sarimax import SARIMAX
from sktime.forecasting.croston import Croston
from sktime.forecasting.fbprophet import Prophet
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.arch import StatsForecastGARCH
from sktime.transformations.series.outlier_detection import HampelFilter

from app.data_managers.namespaces import data_ns
from app.modelling.metrics import mae, rmse, maxae
from app.modelling.models import LinReg, RandomForrest, RegressionTree
from app.modelling.splitters import ExpandingWindowSplitter, SlidingWindowSplitter
from app.modelling.transformers import (
    CompletnessFilter,
    DormantFilter,
    ImputerBackTime,
    ImputerPandas,
    NanDropper,
)
from app.tuner.tuner import Tuner, BEST_MODEL

FORECAST_PERIOD = 24
forecast_horizon = np.arange(1, FORECAST_PERIOD + 1)

c:\Users\Mambo\.conda\envs\bachelor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-13 16:56:05,125	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
forecasters = {
    "PACK_1" : {
        "PROPHET": Prophet(),
        "CROSTON_0.01": Croston(smoothing=0.01),
        "NAIVE_DRIFT": NaiveForecaster(strategy="drift"),
    },
    "PACK_2" : {
        "PROPHET_DEFAULT": Prophet(
            freq="H",
            add_seasonality=True,
            daily_seasonality=True,
            weekly_seasonality=True,
            yearly_seasonality=True,
        ),
        "CROSTON_0.1": Croston(smoothing=0.1),
        "NAIVE_LAST_24H": NaiveForecaster(strategy="last", sp=24),
    },
    "PACK_3" : {
        "PROPHET_MULT_SEAS": Prophet(
            freq="H",
            add_seasonality=True,
            daily_seasonality=True,
            yearly_seasonality=True,
            seasonality_mode="multiplicative",
        ),
        "CROSTON_0.2": Croston(smoothing=0.2),
        "NAIVE_MEAN_30D_SP24H": NaiveForecaster(
            strategy="mean", window_length=30 * 24, sp=24
        ),
    },
    "PACK_4" : {
        "ARIMA_1": ARIMA(),
        "CROSTON_0.5": Croston(smoothing=0.5),
        "NAIVE_MEAN_30D": NaiveForecaster(strategy="mean", window_length=30 * 24),
    },
    "PACK_5": {
        "ARIMA_2": ARIMA((1, 1, 0)),
        "CROSTON_0.8": Croston(smoothing=0.8),
        "NAIVE_MEAN_3D": NaiveForecaster(strategy="mean", window_length=3 * 24),
    },
    "PACK_6" : {
        "LinReg": LinReg(),
        "NAIVE_LAST_48H": NaiveForecaster(strategy="last", sp=48),
        "NAIVE_MEAN_7D": NaiveForecaster(strategy="mean", window_length=7 * 24),
    },
    "PACK_7" : {
        "RandomForrest": RandomForrest(),
        "NAIVE_MEAN_7D_SP24H": NaiveForecaster(
            strategy="mean", window_length=7 * 24, sp=24
        ),
        "StatsForecastGARCH": StatsForecastGARCH(),
    },
    "PACK_8" : {
        "RegressionTree": RegressionTree(),
        "ARIMA_3": ARIMA((1, 1, 1)),
        "SARIMAX_D": SARIMAX((1, 0, 0), (1, 0, 0, 24)),
    }
}

Importing plotly failed. Interactive plots will not work.


In [4]:
splitters = {
    "ExpandingWindowSplitter": ExpandingWindowSplitter(
        fh=forecast_horizon, step_length=FORECAST_PERIOD
    ),
    "SlidingWindowSplitter_FP*60": SlidingWindowSplitter(
        fh=forecast_horizon,
        step_length=FORECAST_PERIOD,
        window_length=FORECAST_PERIOD * 60,
    ),
    "SlidingWindowSplitter_FP": SlidingWindowSplitter(
        fh=forecast_horizon, step_length=FORECAST_PERIOD, window_length=FORECAST_PERIOD
    ),
    "SlidingWindowSplitter_FP*7": SlidingWindowSplitter(
        fh=forecast_horizon, step_length=FORECAST_PERIOD, window_length=FORECAST_PERIOD * 7
    ),
    "SlidingWindowSplitter_FP*30": SlidingWindowSplitter(
        fh=forecast_horizon, step_length=FORECAST_PERIOD, window_length=FORECAST_PERIOD * 30
    ),
}

In [5]:
transformers = [
        DormantFilter(period=FORECAST_PERIOD + 48),
        CompletnessFilter(0.5),
        ImputerBackTime(period_to_take_value_from=24),
        HampelFilter(window_length=72),
        ImputerPandas(method="linear"),
        NanDropper(),
    ]

In [6]:
tuner = Tuner(
    forecasters=forecasters, 
    splitters=splitters,
    metrics={"mae": mae, "rmse": rmse, "maxae": maxae}, 
    strategies={BEST_MODEL: BEST_MODEL, "mean": "mean", "median": "median"},
    dates=["2023-04-20 00", "2023-09-20 00", "2023-12-07 00"],
    input_dir=data_ns.TRANSFORMED_DATA_DIR,
    result_dir=data_ns.FORECAST_RESULT_DIR,
    selection_dir=data_ns.SELECTED_DATA_DIR,
    max_forecasts=4,
    transformers=transformers,
    forecast_horizon=forecast_horizon,
    parallel_batch=10,
    engine="nsga_2",
)

In [8]:
tuner.tune(n_gen=8, n_pop=8, ev_metrics={"rmse": rmse}, result_dir="24h_8gen_8pop_nsga2", time_constraint=0.75)

c:\Users\Mambo\.conda\envs\bachelor\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\Mambo\Desktop\bachelor_migration\app\utils\evaluator.py:109: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  evaluation[f"{dependent_variable}_{str(metric)}"] = float(
c:\Users\Mambo\Desktop\bachelor_migration\app\utils\evaluator.py:109: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  evaluation[f"{dependent_variable}_{str(metric)}"] = float(
c:\Users\Mambo\Desktop\bachelor_migration\app\utils\evaluator.py:109: FutureWarning: Calling float on a s

[array([17.91063755,  7.41302831, 13.62564345,  0.33333333, -0.9537037 ]), array([17.91063755,  7.41302831, 13.62564345,  0.        , -0.9537037 ]), array([17.91063755,  7.41302831, 13.62564345,  0.        , -0.9537037 ]), array([17.91063755,  7.41302831, 13.62564345,  0.33333333, -0.9537037 ])]


c:\Users\Mambo\.conda\envs\bachelor\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\Mambo\Desktop\bachelor_migration\app\utils\evaluator.py:109: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  evaluation[f"{dependent_variable}_{str(metric)}"] = float(
c:\Users\Mambo\Desktop\bachelor_migration\app\utils\evaluator.py:109: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  evaluation[f"{dependent_variable}_{str(metric)}"] = float(
c:\Users\Mambo\Desktop\bachelor_migration\app\utils\evaluator.py:109: FutureWarning: Calling float on a s

[array([17.91063755,  7.41302831, 13.62564345,  0.66666667, -0.9537037 ]), array([17.91063755,  7.41302831, 13.62564345,  0.66666667, -0.9537037 ]), array([17.91063755,  7.41302831, 13.62564345,  0.66666667, -0.9537037 ]), array([17.91063755,  7.41302831, 13.62564345,  0.        , -0.9537037 ])]


c:\Users\Mambo\.conda\envs\bachelor\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\Mambo\Desktop\bachelor_migration\app\utils\evaluator.py:109: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  evaluation[f"{dependent_variable}_{str(metric)}"] = float(
c:\Users\Mambo\Desktop\bachelor_migration\app\utils\evaluator.py:109: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  evaluation[f"{dependent_variable}_{str(metric)}"] = float(
c:\Users\Mambo\Desktop\bachelor_migration\app\utils\evaluator.py:109: FutureWarning: Calling float on a s